In [1]:
import asyncio
import nest_asyncio
from mcp import ClientSession
from mcp.client.sse import sse_client

nest_asyncio.apply()  # Needed to run interactive python

"""
Make sure:
1. The server is running before running this script.
2. The server is configured to use SSE transport.
3. The server is listening on port 8050.

To run the server:
uv run server.py
"""

'\nMake sure:\n1. The server is running before running this script.\n2. The server is configured to use SSE transport.\n3. The server is listening on port 8050.\n\nTo run the server:\nuv run server.py\n'

In [2]:
import asyncio
import json
from contextlib import AsyncExitStack
from typing import Any, Dict, List

import nest_asyncio
from dotenv import load_dotenv
from mcp import ClientSession, StdioServerParameters
from mcp.client.sse import sse_client
from openai import AsyncOpenAI

# Apply nest_asyncio to allow nested event loops (needed for Jupyter/IPython)
nest_asyncio.apply()

# Load environment variables
load_dotenv(".env")

# Global variables to store session state
session = None
exit_stack = AsyncExitStack()
openai_client = AsyncOpenAI()
model = "gpt-5"


In [3]:
import asyncio, os
from contextlib import AsyncExitStack
from mcp import ClientSession
from mcp.client.sse import sse_client

URL = "https://mcp.api.coingecko.com/sse"
HEADERS = {"x-cg-demo-api-key": "CG-TySwLedUWCWnaL3D1eFLo9kd"}

stack = AsyncExitStack()
session: ClientSession | None = None

async def connect():
    global session
    read, write = await stack.enter_async_context(sse_client(URL, headers=HEADERS))
    session = await stack.enter_async_context(ClientSession(read, write))
    await session.initialize()
    return session

async def close():
    await stack.aclose()

session = None
tools = None
async def main():
    global session, tools
    session = await connect()

    tools = await session.list_tools()
    print("Tools:", [t.name for t in tools.tools])

    price = await session.call_tool(
        "get_simple_price",
        {"ids": "bitcoin,ethereum", "vs_currencies": "usd"}
    )
    print("BTC/ETH ->", price.structuredContent or price.content)

    # 🔥 You can still reuse `session` later
    price2 = await asyncio.wait_for(session.call_tool(
                "get_simple_price",
                {"ids": "dogecoin", "vs_currencies": "usd"}
            ),timeout=3)
    print("DOGE ->", price2.structuredContent or price2.content)
    await close()
asyncio.run(main())


Tools: ['get_asset_platforms', 'get_id_coins', 'get_list_coins_categories', 'get_coins_list', 'get_new_coins_list', 'get_coins_markets', 'get_coins_top_gainers_losers', 'get_coins_contract', 'get_range_contract_coins_market_chart', 'get_coins_history', 'get_range_coins_market_chart', 'get_range_coins_ohlc', 'get_global', 'get_id_nfts', 'get_list_nfts', 'get_nfts_market_chart', 'get_onchain_categories', 'get_pools_onchain_categories', 'get_onchain_networks', 'get_networks_onchain_new_pools', 'get_network_networks_onchain_new_pools', 'get_networks_onchain_trending_pools', 'get_network_networks_onchain_trending_pools', 'get_networks_onchain_dexes', 'get_pools_networks_onchain_dexes', 'get_networks_onchain_pools', 'get_address_networks_onchain_pools', 'get_pools_networks_onchain_info', 'get_timeframe_pools_networks_onchain_ohlcv', 'get_pools_networks_onchain_trades', 'get_address_networks_onchain_tokens', 'get_tokens_networks_onchain_info', 'get_tokens_networks_onchain_top_holders', 'get_t

In [4]:
# def get_mcp_tools() -> List[Dict[str, Any]]:
#     """Get available tools from the MCP server in OpenAI format.

#     Returns:
#         A list of tools in OpenAI format.
#     """
#     return [
#         {
#             "type": "function",
#             "function": {
#                 "name": tool.name,
#                 "description": tool.description,
#                 "parameters": tool.inputSchema,
#             },
#         }
#         for tool in tools.tools
#     ]

# tools = get_mcp_tools()
# tools
tools = [
    {
        "type": "function",
        "function": {
            "name": tool.name,
            "description": tool.description,
            "parameters": tool.inputSchema,
        },
    }
    for tool in tools.tools
]
tools

[{'type': 'function',
  'function': {'name': 'get_asset_platforms',
   'description': 'This endpoint allows you to **query all the asset platforms on CoinGecko**',
   'parameters': {'type': 'object',
    'properties': {'filter': {'type': 'string',
      'description': 'apply relevant filters to results',
      'enum': ['nft']}}}}},
 {'type': 'function',
  'function': {'name': 'get_id_coins',
   'description': 'This endpoint allows you to **query all the metadata (image, websites, socials, description, contract address, etc.) and market data (price, ATH, exchange tickers, etc.) of a coin from the CoinGecko coin page based on a particular coin ID**',
   'parameters': {'type': 'object',
    'properties': {'id': {'type': 'string'},
     'community_data': {'type': 'boolean',
      'description': 'include community data, default: true'},
     'developer_data': {'type': 'boolean',
      'description': 'include developer data, default: true'},
     'dex_pair_format': {'type': 'string',
      '

In [5]:
system_prompt = """
You have the access to CoinGecko MCP Server, to find the cryptomarket trends. 
Based on the user data and user query, 
you need to find all tools that need to be called to address the user query.
"""

user_data = "USDC BALANCE: 23000"
user_query = "I want to know curent trends. Please analyze, should I hold my asset right now, or what?"
user_prompt = f"""
==> USER DATA:
{user_data}

==> USER QUERY:
{user_query}"""

response = await openai_client.chat.completions.create(
        model=model,
        messages=[
                {"role": "system","content": system_prompt},
                {"role": "user", "content": user_prompt}],
        tools=tools,
        tool_choice="auto")
assistant_message = response.choices[0].message
print(assistant_message)
payload_response = {}
i = 1

session = await connect()
for tool_call in assistant_message.tool_calls:
    args = json.loads(tool_call.function.arguments)
    result = await session.call_tool(tool_call.function.name, arguments=args)
    args["function_name"] = tool_call.function.name
    args["tool_call_result"] = json.loads(result.content[0].text)
    payload_response[f"tool call - {i}"]=args
    print(result)
    i += 1
await close()

ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=[ChatCompletionMessageFunctionToolCall(id='call_kP53UK8fcuD32BCSkR1SxCGi', function=Function(arguments='{}', name='get_global'), type='function'), ChatCompletionMessageFunctionToolCall(id='call_MEmPuPubwMBeKUBFStXhrLHX', function=Function(arguments='{"vs_currency": "usd", "order": "market_cap_desc", "per_page": 10, "page": 1, "price_change_percentage": "1h,24h,7d"}', name='get_coins_markets'), type='function'), ChatCompletionMessageFunctionToolCall(id='call_1dboQJ8G4hrefjtxSr7kVXH5', function=Function(arguments='{"vs_currency": "usd", "duration": "24h", "top_coins": "1000"}', name='get_coins_top_gainers_losers'), type='function'), ChatCompletionMessageFunctionToolCall(id='call_1gk2Wlf9F2qLAkXUAV05BEdZ', function=Function(arguments='{"vs_currency": "usd", "duration": "7d", "top_coins": "1000"}', name='get_coins_top_gainers_losers'), type='function'), ChatComplet

In [6]:
f"{payload_response}"

'{\'tool call - 1\': {\'function_name\': \'get_global\', \'tool_call_result\': {\'data\': {\'active_cryptocurrencies\': 18409, \'upcoming_icos\': 0, \'ongoing_icos\': 49, \'ended_icos\': 3376, \'markets\': 1353, \'total_market_cap\': {\'btc\': 35167463.8460431, \'eth\': 874702192.7317694, \'ltc\': 35090098206.86775, \'bch\': 7167347138.244646, \'bnb\': 4596102222.985339, \'eos\': 8009415028216.106, \'xrp\': 1328477447262.9536, \'xlm\': 9945063515711.662, \'link\': 167496616952.46152, \'dot\': 1025202270999.9907, \'yfi\': 713816997.1594315, \'sol\': 20574129487.10071, \'usd\': 3867876477825.285, \'aed\': 14204776364813.357, \'ars\': 5256557848933008, \'aud\': 5968400288761.382, \'bdt\': 472530462791605.94, \'bhd\': 1458344147199.2456, \'bmd\': 3867876477825.285, \'brl\': 20928306046217.055, \'cad\': 5359511437869.174, \'chf\': 3119326343071.758, \'clp\': 3723333933848951.5, \'cny\': 27686646615921.176, \'czk\': 81716235691490.55, \'dkk\': 24840477445466.395, \'eur\': 3327959600285.6533,

In [7]:
from openai import OpenAI
gpt_client = OpenAI()

def gpt_response(messages):
    response = gpt_client.responses.create(
        model="gpt-5",
        input=messages,
        text={
            "format": {
            "type": "text"
            },
            "verbosity": "medium"
        },
        reasoning={
            "effort": "medium"
        },
        tools=[],
        store=True
        )
    return response.output[1].content[0].text

In [8]:
response = gpt_response([
    {
        "role":"system",
        "content":"""
            Given the CoinGecko Server data from function calling, 
            which might be relevant to address the user query,
            you need to give the response of your recommendation to the user by carefully analyzing the data provided from function calling"""
    },
    {
        "role":"user",
        "content":f"""
        ==> FUNCTION CALLING RESULT:
        {payload_response}

        ==> USER DATA:
        {user_data}

        ==> USER QUERY:
        {user_query}"""
    }
])

In [9]:
print(response.output[1].content[0].text)

AttributeError: 'str' object has no attribute 'output'